# BERT를 사용한 문서 분류

https://situdy.tistory.com/70

In [1]:
import pandas as pd
import numpy as np
import random
import time
import datetime
from tqdm import tqdm

import csv
import os

import tensorflow as tf
import torch

# BERT 사용을 위함
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, BertConfig, AdamWeightDecay
from transformers import get_linear_schedule_with_warmup

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# for padding
from tensorflow.keras.preprocessing.sequence import pad_sequences 

# 전처리 및 평가 지표
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, hamming_loss

In [2]:
import pandas as pd
df = pd.read_csv('./data/df_all_preprocessed_noun.csv')
df.head()

,url,press,title,datetime,text,label,nouns
0,https://www.yna.co.kr/view/AKR2025010104210000...,연합뉴스,"이재명, 대권주자 적합도 30%대 '독주'…與후보들은 한 자릿수",2025-01-01 16:38:15+09:00,설승은기자 구독 구독중 이전 다음\n\n언론사 세 곳 신년 여론조사…홍준표·오세훈·...,이재명,"['설', '승', '기자', '구독', '구독', '중', '다음', '언론사',..."
1,https://www.hani.co.kr/arti/politics/politics_...,한겨레,국민 10명 중 7명 “윤석열 파면해야”…차기 대선주자 이재명 1위,NaN,본문\n\n기사를 읽어드립니다 Your browser does not support...,이재명,"['본문', '기사', '일', '서울', '종로구', '헌법재판소', '헌재', ..."
2,https://www.seoul.co.kr/news/society/2025/01/0...,서울신문,"“바쁘실 텐데 1분만”…이재명 붙잡은 유족, 눈물 흘리며 전한 말은",2025-01-01 00:00:00,이미지 확대 유가족 요구사항 메모하는 이재명 대표 더불어민주당 이재명 대표가 31일...,이재명,"['이미지', '확대', '유가족', '요구사항', '메모', '이재명', '대표'..."
3,https://news.tvchosun.com/site/data/html_dir/2...,TV조선,"이재명 ""절망의 늪에 빠진 국민의 삶에 함께하겠다""",2025-01-01 00:00:00,이재명 더불어민주당 대표 /연합뉴스\n\n이재명 더불어민주당 대표는 1일 신년을 맞...,이재명,"['이재명', '더불어민주당', '대표', '연합뉴스', '이재명', '더불어민주당..."
4,http://mbn.mk.co.kr/pages/news/newsView.php?ca...,MBN,"이재명 신년사 ""어둠 깊을수록 '새로운 나라' 소망 선명해져""",2025-01-01 10:01:00+09:00,"""절망의 늪 빠진 국민의 삶 함께 하겠다""\n\n""우리 앞의 비극·고난 극복하고 새...",이재명,"['절망', '늪', '국민', '삶', '우리', '앞', '비극', '고난', ..."


In [3]:
df['label'] = df['label'].replace(['윤석열', '이재명'],[0, 1])
df.head()

C:\Users\dmlab\AppData\Local\Temp\ipykernel_19620\539941490.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['label'] = df['label'].replace(['윤석열', '이재명'],[0, 1])


,url,press,title,datetime,text,label,nouns
0,https://www.yna.co.kr/view/AKR2025010104210000...,연합뉴스,"이재명, 대권주자 적합도 30%대 '독주'…與후보들은 한 자릿수",2025-01-01 16:38:15+09:00,설승은기자 구독 구독중 이전 다음\n\n언론사 세 곳 신년 여론조사…홍준표·오세훈·...,1,"['설', '승', '기자', '구독', '구독', '중', '다음', '언론사',..."
1,https://www.hani.co.kr/arti/politics/politics_...,한겨레,국민 10명 중 7명 “윤석열 파면해야”…차기 대선주자 이재명 1위,NaN,본문\n\n기사를 읽어드립니다 Your browser does not support...,1,"['본문', '기사', '일', '서울', '종로구', '헌법재판소', '헌재', ..."
2,https://www.seoul.co.kr/news/society/2025/01/0...,서울신문,"“바쁘실 텐데 1분만”…이재명 붙잡은 유족, 눈물 흘리며 전한 말은",2025-01-01 00:00:00,이미지 확대 유가족 요구사항 메모하는 이재명 대표 더불어민주당 이재명 대표가 31일...,1,"['이미지', '확대', '유가족', '요구사항', '메모', '이재명', '대표'..."
3,https://news.tvchosun.com/site/data/html_dir/2...,TV조선,"이재명 ""절망의 늪에 빠진 국민의 삶에 함께하겠다""",2025-01-01 00:00:00,이재명 더불어민주당 대표 /연합뉴스\n\n이재명 더불어민주당 대표는 1일 신년을 맞...,1,"['이재명', '더불어민주당', '대표', '연합뉴스', '이재명', '더불어민주당..."
4,http://mbn.mk.co.kr/pages/news/newsView.php?ca...,MBN,"이재명 신년사 ""어둠 깊을수록 '새로운 나라' 소망 선명해져""",2025-01-01 10:01:00+09:00,"""절망의 늪 빠진 국민의 삶 함께 하겠다""\n\n""우리 앞의 비극·고난 극복하고 새...",1,"['절망', '늪', '국민', '삶', '우리', '앞', '비극', '고난', ..."


In [4]:
del df['url'], df['datetime'], df['nouns']

In [5]:
df.to_csv('news_data.csv', index=False, encoding='utf-8-sig')

In [6]:
from datasets import load_dataset

all_data = load_dataset(
        "csv",
        data_files={
            "train": "news_data.csv",
        },
    )


Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
all_data

DatasetDict({
    train: Dataset({
        features: ['press', 'title', 'text', 'label'],
        num_rows: 15662
    })
})

In [8]:
cs = all_data['train'].train_test_split(0.2)
train_cs = cs["train"]
test_cs = cs["test"]

In [9]:
# 훈련 데이터를 다시 8:2로 분리 후 훈련 데이터와 검증 데이터로 저장
cs = train_cs.train_test_split(0.2)
train_cs = cs["train"]
valid_cs = cs["test"]

In [10]:
print('두번째 샘플 출력 :', train_cs['text'][1])
print('두번째 샘플의 레이블 출력 :', train_cs['label'][1])

두번째 샘플 출력 : [서울=뉴시스] 조성봉 기자= 우원식 국회의장이 17일 오후 서울 여의도 국회에서 열린 제421회 국회(임시회) 제1차 본회의에서 윤석열 정부의 내란ㆍ외환 행위의 진상규명을 위한 특별검사 임명 등에 관한 법률안에 대한 수정안 가결을 선언하자 국민의힘 의원들이 자리에서 일어나 밖으로 나서고 있다. 2025.01.17. [email protected]
두번째 샘플의 레이블 출력 : 1


In [11]:
# 훈련 데이터, 검증 데이터, 테스트 데이터에 대해서 `[CLS] 문장 [SEP]` 구조를 만듭니다.

train_sentences = list(map(lambda x: '[CLS] ' + str(x) + ' [SEP]', train_cs['text']))
validation_sentences = list(map(lambda x: '[CLS] ' + str(x) + ' [SEP]', valid_cs['text']))
test_sentences = list(map(lambda x: '[CLS] ' + str(x) + ' [SEP]', test_cs['text']))

In [12]:
train_labels = train_cs['label']
validation_labels = valid_cs['label']
test_labels = test_cs['label']

In [13]:
test_sentences[0]


"[CLS] 탄핵 반대 집회/사진 이용우 기자\n\n헌법재판소가 윤석열 대통령 탄핵심판 선고일 통보를 앞두고 있는 가운데 서울 도심 곳곳에서 찬반 집회가 열릴 예정이다.\n\n서울경찰청은 29일 오후 서울 도심권에서 퇴진비상행동과 자유통일당 등 수만 명이 주최하는 집회와 행진이 개최된다고 밝혔다.\n\n윤석열 즉각 퇴진·사회대개혁 비상행동은 이날 오후 5시 광화문 동십자각에서 '제17차 범시민 대행진'을 연다. 경찰 신고 인원은 10만명이다. 이들은 종로구 사직로, 율곡로 일대에서 집회를 열고, 종로방향으로 행진할 예정이다.\n\n보수단체 세이브코리아는 이날 오후 1시부터 서울 영등포구 여의대로에서 '국가비상기도회'를 진행한다. 전광훈 사랑제일교회 목사가 이끄는 자유통일당과 대한민국바로세우기운동본부도 종로구 동화면세점 앞에서 윤 대통령 탄핵 반대 집회를 연다.\n\n두 단체 신고 인원은 22만 명이다.\n\n경찰은 집회·행진 구간 주변에 교통경찰 220여 명을 배치한다. 시민 불편을 최소화하기 위해 차량 우회 등 교통소통 관리에 나선다. [SEP]"

In [14]:
# 한국어 BERT 중 하나인 'klue/bert-base'를 사용.
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')

In [15]:
MAX_LEN = 128

def data_to_tensor (sentences, labels):
  # 정수 인코딩 과정. 각 텍스트를 토큰화한 후에 Vocabulary에 맵핑되는 정수 시퀀스로 변환한다.
  # ex) ['안녕하세요'] ==> ['안', '녕', '하세요'] ==> [231, 52, 45]
  tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
  input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

  # pad_sequences는 패딩을 위한 모듈. 주어진 최대 길이를 위해서 뒤에서 0으로 채워준다.
  # ex) [231, 52, 45] ==> [231, 52, 45, 0, 0, 0]
  input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post") 

  attention_masks = []

  for seq in input_ids:
      seq_mask = [float(i > 0) for i in seq]
      attention_masks.append(seq_mask)

  tensor_inputs = torch.tensor(input_ids)
  tensor_labels = torch.tensor(labels)
  tensor_masks = torch.tensor(attention_masks)

  return tensor_inputs, tensor_labels, tensor_masks

In [16]:
train_inputs, train_labels, train_masks = data_to_tensor(train_sentences, train_labels)
validation_inputs, validation_labels, validation_masks = data_to_tensor(validation_sentences, validation_labels)
test_inputs, test_labels, test_masks = data_to_tensor(test_sentences, test_labels)

In [17]:
print(train_inputs[0])
print(train_masks[0])

tensor([    2,  1865,  2115,  2557,    12,    83, 24728, 31415,    13,  4989,
         4133, 11187,  3951,  3669,  2154,  6283,  2530,  6233,  2525, 11059,
         2125,  7734,  2069,  3923,  2205,  2259,   842,  2170,  1513,  2051,
         4389,  2897,  4228,  2069,  1331,  2051,  2223,  2259,   575,  2069,
        16519,    18,    37, 23548,  3611,  2052,    38,  2170,  2318,  1865,
         2115,  2557,  6283,  2069,  3644, 16246,  2371,  4683,  1902,  2069,
          904,    16,    37,  2079,  1041,  2154,   881,  2088,  3748,  2470,
         4150,  2069,  5740,  4795,  4045,  2145,  4991,  3656,  3731,  2138,
        20736, 11295,    38,  2259,    37,  2170,  2318,  8221,  2085,   575,
        28674,    18,  3696,  5107,    37,  2116,  6441,  4555,  2073,  1198,
         2205,  2062,    18,   636,  2116, 22122,  2470,  3611,  2052, 19669,
          116,   717,  2259,  5352,  2069,  1041,  2371,  2051,     5,   117,
         3609,  1892,   575, 28674,    18,  5738,  3629,  2079],

In [18]:
batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [19]:
print('훈련 데이터의 크기:', len(train_labels))
print('검증 데이터의 크기:', len(validation_labels))
print('테스트 데이터의 크기:', len(test_labels))

훈련 데이터의 크기: 10023
검증 데이터의 크기: 2506
테스트 데이터의 크기: 3133


In [20]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

No GPU available, using the CPU instead.


In [21]:
num_labels = 2

model = BertForSequenceClassification.from_pretrained("klue/bert-base", num_labels=num_labels)
# model.cuda()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
# 옵티마이저 선택
optimizer = torch.optim.AdamW(model.parameters(),
                  lr = 2e-5,
                  eps = 1e-8
                )

In [23]:
# 몇 번의 에포크(전체 데이터에 대한 학습 횟수)를 할 것인지 선택
epochs = 2
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [24]:
def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))  # hh:mm:ss

In [25]:
def metrics(predictions, labels):
    y_pred = predictions
    y_true = labels

    # 사용 가능한 메트릭들을 사용한다.
    accuracy = accuracy_score(y_true, y_pred)
    f1_macro_average = f1_score(y_true=y_true, y_pred=y_pred, average='macro', zero_division=0)
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro', zero_division=0)
    f1_weighted_average = f1_score(y_true=y_true, y_pred=y_pred, average='weighted', zero_division=0)

    # 메트릭 결과에 대해서 리턴
    metrics = {'accuracy': accuracy,
               'f1_macro': f1_macro_average,
               'f1_micro': f1_micro_average,
               'f1_weighted': f1_weighted_average}

    return metrics

In [26]:
# 랜덤 시드값.
seed_val = 777
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

model.zero_grad()
for epoch_i in range(0, epochs):
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    t0 = time.time()
    total_loss = 0

    model.train()

    for step, batch in tqdm(enumerate(train_dataloader)):
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        loss = outputs[0]
        total_loss += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # gradient clipping if it is over a threshold
        optimizer.step()
        scheduler.step()

        model.zero_grad()

    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.4f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))

======== Epoch 1 / 2 ========


314it [40:56,  7.82s/it]



  Average training loss: 0.2670
  Training epcoh took: 0:40:56
======== Epoch 2 / 2 ========


314it [41:12,  7.87s/it]


  Average training loss: 0.1906
  Training epcoh took: 0:41:13


In [27]:
t0 = time.time()
model.eval()
accum_logits, accum_label_ids = [], []

for batch in validation_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    for b in logits:
        # 3개의 값 중 가장 큰 값을 예측한 인덱스로 결정
        # ex) [ 3.5134246  -0.30875662 -2.111316  ] ==> 0
        accum_logits.append(np.argmax(b))

    for b in label_ids:
        accum_label_ids.append(b)

accum_logits = np.array(accum_logits)
accum_label_ids = np.array(accum_label_ids)
results = metrics(accum_logits, accum_label_ids)

print("Accuracy: {0:.4f}".format(results['accuracy']))
print("F1 (Macro) Score: {0:.4f}".format(results['f1_macro']))
print("F1 (Micro) Score: {0:.4f}".format(results['f1_micro']))
print("F1 (Weighted) Score: {0:.4f}".format(results['f1_weighted']))

Accuracy: 0.9210
F1 (Macro) Score: 0.9209
F1 (Micro) Score: 0.9210
F1 (Weighted) Score: 0.9210


In [28]:
t0 = time.time()
model.eval()
accum_logits, accum_label_ids = [], []

for step, batch in tqdm(enumerate(test_dataloader)):
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    for b in logits:
        # 3개의 값 중 가장 큰 값을 예측한 인덱스로 결정
        # ex) [ 3.5134246  -0.30875662 -2.111316  ] ==> 0
        accum_logits.append(np.argmax(b))

    for b in label_ids:
        accum_label_ids.append(b)

accum_logits = np.array(accum_logits)
accum_label_ids = np.array(accum_label_ids)
results = metrics(accum_logits, accum_label_ids)

print("Accuracy: {0:.4f}".format(results['accuracy']))
print("F1 (Macro) Score: {0:.4f}".format(results['f1_macro']))
print("F1 (Micro) Score: {0:.4f}".format(results['f1_micro']))
print("F1 (Weighted) Score: {0:.4f}".format(results['f1_weighted']))

98it [03:25,  2.10s/it]

Accuracy: 0.9167
F1 (Macro) Score: 0.9166
F1 (Micro) Score: 0.9167
F1 (Weighted) Score: 0.9167


In [29]:
from transformers import pipeline

In [30]:
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0, max_length=512, #model=model.cuda(), 
                return_all_scores=True, function_to_apply='softmax')

Device set to use cpu
C:\Users\dmlab\anaconda3\Lib\site-packages\transformers\pipelines\text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [31]:
text = '윤석열 대통령이 탄핵될 것인가?'

result = pipe(text)
print(result)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[[{'label': 'LABEL_0', 'score': 0.9185092449188232}, {'label': 'LABEL_1', 'score': 0.08149071782827377}]]
